In [1]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_FRACDIFF, ALL_RAW_FEAT, SSMContainer
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles("2025-01-01", "2025-10-12")

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_full.index.to_numpy() < date_to_timestamp("2025-10-12"))
]

PyTorch configured: device=cpu, dtype=torch.float32


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


getting raw features...
df_raw_feats.shape = (2985, 9108)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [35]:
import json

MODEL_TYPE = "r"
LAG = 6
PRED_NEXT = 3
THRESHOLD = 0
LOWER_BOUND = -1
UPPER_BOUND = 1
MODEL_NAME = f"{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}"

In [36]:
with open("strategies/BinanceBtcDeapV1Voting/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

close_price = candles[:, 2][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")
assert len(datetimes) == len(close_price)

model = lgb.Booster(model_file=f"strategies/BinanceBtcDeapV1Voting/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [37]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    close_price,
    PRED_NEXT,
    output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
    granularity=0.02,
    lower_bound=LOWER_BOUND,
    upper_bound=UPPER_BOUND,
    threshold=THRESHOLD,
)


⚠️ 警告：发现 67 个超出范围 [-1, 1] 的值
  - 占比: 3.31%
  - 实际范围: [-1.3533, 1.3119]
  - 超出范围的值将被归类到最近的边界切片


📊 时序对齐信息:
  - pred_next: 3 (预测未来第3根K线)
  - 原始数据长度: 2026
  - 对齐后数据长度: 2023
  - 丢弃的尾部数据: 3 条

已生成: slice_0.9800_1.0000_ratio_0.0015.jpg (切片区间: [0.9800, 1.0000] (做多), 样本占比: 0.15%)
已生成: slice_0.9600_0.9800_ratio_0.0030.jpg (切片区间: [0.9600, 0.9800] (做多), 样本占比: 0.30%)
已生成: slice_0.9400_0.9600_ratio_0.0010.jpg (切片区间: [0.9400, 0.9600] (做多), 样本占比: 0.10%)
已生成: slice_0.9200_0.9400_ratio_0.0025.jpg (切片区间: [0.9200, 0.9400] (做多), 样本占比: 0.25%)
已生成: slice_0.9000_0.9200_ratio_0.0020.jpg (切片区间: [0.9000, 0.9200] (做多), 样本占比: 0.20%)
已生成: slice_0.8800_0.9000_ratio_0.0035.jpg (切片区间: [0.8800, 0.9000] (做多), 样本占比: 0.35%)
已生成: slice_0.8600_0.8800_ratio_0.0020.jpg (切片区间: [0.8600, 0.8800] (做多), 样本占比: 0.20%)
已生成: slice_0.8400_0.8600_ratio_0.0025.jpg (切片区间: [0.8400, 0.8600] (做多), 样本占比: 0.25%)
已生成: slice_0.8200_0.8400_ratio_0.0049.jpg (切片区间: [0.8200, 0.8400] (做多), 样本占比: 0.49%)
已生成: slice_0.8000_0.8200_ratio_0.0049.jpg (切片区

In [36]:
from strategies.BinanceBtcDeapV1Voting.models.config import LGBMContainer

mc = LGBMContainer(
    model_type=MODEL_TYPE,
    lag=LAG,
    pred_next=PRED_NEXT,
    threshold=THRESHOLD,
)
print(mc.MODEL_NAME)

c_L3_N2


In [32]:
# 根据图片设定合适的filters

GIVEUP_FILTERS = [
    (0, 0.02),
    (0.12, 0.14),
    (0.24, 0.26),
    (0.32, 0.38),
    (0.4, 0.42),
    (0.46, 0.48),
    (0.5, 0.52),
]

REVERSE_FILTERS = [
    (0.04, 0.08),
    (0.16, 0.18),
    (0.2, 0.24),
    (0.28, 0.3),
    (0.48, 0.5),
    (0.52, 0.54),
]

for f in GIVEUP_FILTERS:
    mc.add_giveup_filter(f[0], f[1])

for f in REVERSE_FILTERS:
    mc.add_reverse_filter(f[0], f[1])

In [34]:
mc.save_filters()

Filters saved to /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/model_c_L3_N1_filters.json
